In [ ]:
import os
import time
import datetime
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 
driver = webdriver.Firefox()

In [ ]:
playlist = 'https://open.spotify.com/playlist/37i9dQZF1DX0FOF1IUWK1W'\
driver.get(playlist)

In [ ]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

df = pd.DataFrame(columns=['musica'])
links = soup.find_all('button')
for link in links:
    href = link.get('aria-label')
    if 'Tocar' in str(href): 
        df = df.append({'musica': str(href).replace('Tocar ', '',)}, ignore_index=True)
        #print(href)
df = df[~df['musica'].str.contains('Top Brasil')]
df['musica'] = df['musica'].str.replace('- Ao Vivo ', ' ')
df

In [ ]:
columns_names = ['musica', 'titulo','artista', 'letra']
df_letra = pd.DataFrame(columns=columns_names)
df_letra

for index, row in df.iterrows():
    consulta = row['musica']
    titulo= ''
    artista = ''
    letra = ""
    try:
        url = 'https://www.letras.mus.br'
        driver.get(url)
        driver.find_element(By.XPATH, '//*[@id="main_suggest"]').clear()
        driver.find_element(By.XPATH, '//*[@id="main_suggest"]').send_keys(consulta)
        driver.find_element(By.XPATH, '//*[@id="main_suggest"]').send_keys(Keys.ENTER)
        time.sleep(2)
        driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[1]/div[3]/div/div/div/div/div/div/div/div[5]/div[2]/div/div/div[1]/div[1]/div[1]/div[1]/div/a').click()

        time.sleep(2)
        titulo = driver.find_element(By.XPATH, '//*[@id="js-lyric-cnt"]/article/div[1]/div[2]/h1').get_attribute('innerHTML') #titulo
        artista = driver.find_element(By.XPATH, '//*[@id="js-lyric-cnt"]/article/div[1]/div[2]/h2/a/span').get_attribute('innerHTML') #artista

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        target_div = soup.find('div', class_='cnt-letra')
        paragraphs = target_div.find_all('p')
        
        for p in paragraphs:
            for br in p.find_all('br'):
                br.replace_with('\n')
            paragraph_text = p.text
            letra +=  paragraph_text + '\n' + '\n'
        list_row = [consulta, titulo, artista, letra]
        df_letra.loc[len(df_letra)] = list_row 
    except:
        list_row = [consulta, titulo, artista, letra]
        df_letra.loc[len(df_letra)] = list_row 
        continue

df_letra

In [ ]:
df_letra.to_csv("df_letra.csv")
df.to_csv("df_top50_musicas_nacionais.csv")